<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import dataset</a></span></li><li><span><a href="#Embeddings-CTNet-(same-for-other-models)-Conv2_layer----All-graphs" data-toc-modified-id="Embeddings-CTNet-(same-for-other-models)-Conv2_layer----All-graphs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Embeddings CTNet (same for other models) Conv2_layer -- All graphs</a></span><ul class="toc-item"><li><span><a href="#Modificar-modelo-para-sacar-cosas" data-toc-modified-id="Modificar-modelo-para-sacar-cosas-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Modificar modelo para sacar cosas</a></span></li></ul></li><li><span><a href="#Table-of-percentage-of-added/removed-edges" data-toc-modified-id="Table-of-percentage-of-added/removed-edges-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Table of percentage of added/removed edges</a></span></li><li><span><a href="#Rewiring-plots-and" data-toc-modified-id="Rewiring-plots-and-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Rewiring plots and</a></span></li><li><span><a href="#All-graphs-embedding" data-toc-modified-id="All-graphs-embedding-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>All graphs embedding</a></span></li><li><span><a href="#DIGL" data-toc-modified-id="DIGL-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>DIGL</a></span></li></ul></div>

Para CTNet, MinCutNet, GapNet and DIGL (4 figuras de cada task): 
- [ ] Embeddings de un grafo solo *conv2* -> dispersión
    * Seleccionar grafo que tenga buen accuracy y cambie más la predicción respecto a baseline
    * Train vs test
- [X] Embeddings de to los grafos (después del readout) pa plotearlo todos juntos y que se vea la separación entre clases
    * Train vs test
- [ ] Grafo con ejes con peso CT matrix en CTNet
    * Seleccionar grafo que tenga buen accuracy y cambie más la predicción respecto a baseline
    * Train vs test
- [ ] Grafo con ejes adjacency modificada en GapNet
    * Seleccionar grafo que tenga buen accuracy y cambie más la predicción respecto a baseline
    * Train vs test
- [X] Datos sobre grafos

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from nets import CTNet, GAPNet

import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from transform_features import FeatureDegree
from torch_geometric.datasets import TUDataset

from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import DenseGraphConv
from torch_geometric.utils import to_dense_batch, to_dense_adj
from CT_layer import dense_CT_rewiring
from MinCut_Layer import dense_mincut_pool
from sklearn.model_selection import train_test_split

import torch.nn as nn
import warnings

import networkx as nx

warnings.filterwarnings('ignore')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
@torch.no_grad()
def test(modelo, loader, device):
    modelo.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        pred, mc_loss, o_loss = modelo(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()

    return loss, correct / len(loader.dataset)


## Import dataset

In [5]:
dataset = TUDataset(root='data_colab/TUDataset',name="REDDIT-BINARY", pre_transform=FeatureDegree(), use_node_attr=True)
BATCH_SIZE = 16
num_of_centers = 420

"""dataset = TUDataset(root='data_colab/TUDataset',name="MUTAG")
BATCH_SIZE = 32
num_of_centers = 17"""

'dataset = TUDataset(root=\'data_colab/TUDataset\',name="MUTAG")\nBATCH_SIZE = 32\nnum_of_centers = 17'

In [6]:
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}, {dataset.num_node_features}, {dataset.num_node_attributes}')
print(f'Number of classes: {dataset.num_classes}')

print()
datum = dataset[0]  # Get the first graph object.
print(datum)
print('=============================================================')
# Gather some statistics about the first graph.
print(f'Number of nodes: {datum.num_nodes}')
print(f'Number of edges: {datum.num_edges}')
print(f'Average node degree: {datum.num_edges / datum.num_nodes:.2f}')
print(f'Has isolated nodes: {datum.has_isolated_nodes()}')
print(f'Has self-loops: {datum.has_self_loops()}')
print(f'Is undirected: {datum.is_undirected()}')


print(dataset)

Dataset: REDDIT-BINARY(2000):
Number of graphs: 2000
Number of features: 1, 1, 1
Number of classes: 2

Data(edge_index=[2, 480], y=[1], x=[218, 1], num_nodes=218)
Number of nodes: 218
Number of edges: 480
Average node degree: 2.20
Has isolated nodes: False
Has self-loops: False
Is undirected: True
REDDIT-BINARY(2000)


In [7]:
train_indices, test_indices = train_test_split(list(range(len(dataset.data.y))), test_size=0.2, stratify=dataset.data.y,
                                random_state=12345, shuffle=True)

train_indices.extend(test_indices)
new_order = train_indices
dataset = dataset[new_order]
loader =  DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

## Embeddings CTNet (same for other models) Conv2_layer -- All graphs

**Probar modelo guardaddo**

In [8]:
if dataset.name=='REDDIT-BINARY':
    modelito = "models/REDDIT-BINARY_CTNet_iter0.pth"
elif dataset.name=='MUTAG':
    modelito = "models/MUTAG_CTNet_iter0.pth"
else:
    raise Exception('No model for that dataset')

In [ ]:
#Epoch: 059, Train Loss: 165.455, Train Acc: 0.711, Test Loss: 150.841, Test Acc: 0.750
model =  CTNet(dataset.num_features, dataset.num_classes, k_centers=num_of_centers).to(device)
model.load_state_dict(torch.load(modelito, map_location=torch.device(device)))
model.eval()

In [ ]:
loss, acc = test(model, loader, device)

In [ ]:
loss, acc

### Modificar modelo para sacar cosas

**Modelo y Test modificado que coja los embeddings**

In [ ]:
class CTNet_embedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, k_centers, hidden_channels=32):
        super(CTNet_embedding, self).__init__()
    
        self.lin1 = Linear(in_channels, hidden_channels)
        num_of_centers1 =  k_centers # k1 #order of number of nodes
        self.pool1 = Linear(hidden_channels, num_of_centers1)
        #self.CT = CTLayer()
        self.conv1 = DenseGraphConv(hidden_channels, hidden_channels)
        num_of_centers2 =  16 # k2 #mincut 
        self.pool2 = Linear(hidden_channels, num_of_centers2)
        #self.MinCut = MinCutLayer()
        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels) # MLPs towards out 
        self.lin3 = Linear(hidden_channels, out_channels)
        
        self.readout = torch.zeros(0)#Creamos la variable que recoge nuestros embedings

    def forward(self, x, edge_index, batch):    # x torch.Size([N, N]),  data.batch  torch.Size([661])  
        # Make all adjacencies of size NxN 
        adj = to_dense_adj(edge_index, batch) # adj torch.Size(B, N, N])
        # Make all x_i of size N=MAX(N1,...,N20), e.g. N=40:
        x, mask = to_dense_batch(x, batch) # x torch.Size([20, N, 32]) ; mask torch.Size([20, N]) batch_size=20

        x = self.lin1(x)
        # First mincut pool for computing Fiedler adn rewire 
        s1  = self.pool1(x)

        if torch.isnan(adj).any():
            print("adj nan")
        if torch.isnan(x).any():
            print("x nan")
        
        # CT REWIRING
        adj, CT_loss, ortho_loss1 = dense_CT_rewiring(x, adj, s1, mask) # out: x torch.Size([20, N, F'=32]),  adj torch.Size([20, N, N])

        # CONV1: Now on x and rewired adj: 
        x = self.conv1(x, adj) #out: x torch.Size([20, N, F'=32])

        # MLP of k=16 outputs s
        s2 = self.pool2(x) # s torch.Size([20, N, k])
        
        # MINCUT_POOL
        x, adj, mincut_loss2, ortho_loss2 = dense_mincut_pool(x, adj, s2, mask) # out x torch.Size([20, k=16, F'=32]),  adj torch.Size([20, k2=16, k2=16])

        # CONV2: Now on coarsened x and adj: 
        x = self.conv2(x, adj) #out x torch.Size([20, 16, 32])
        
        # Readout for each of the 20 graphs
        x = x.sum(dim=1) # x torch.Size([20, 32])
        
        #Queremos esta x, por lo que nos la guardamos
        self.readout = x.clone()
        
        # Final MLP for graph classification: hidden channels = 32
        x = F.relu(self.lin2(x)) # x torch.Size([20, 32])
        x = self.lin3(x) #x torch.Size([20, 2])
        #print(x.shape)
        
        CT_loss = CT_loss + ortho_loss1
        mincut_loss = mincut_loss2 + ortho_loss2
        return F.log_softmax(x, dim=-1), CT_loss, mincut_loss


In [ ]:
model =  CTNet_embedding(dataset.num_features, dataset.num_classes, k_centers=num_of_centers).to(device)
model.load_state_dict(torch.load(modelito, map_location=torch.device(device)))
model.eval()

In [ ]:
@torch.no_grad()
def test_embedd(modelo, loader, device):
    test_predictions = []
    test_labels = []
    modelo.eval()
    correct = 0
    for i,data in enumerate(loader):
        data = data.to(device)
        pred, mc_loss, o_loss = modelo(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()
        test_predictions.extend(pred.max(dim=1)[1].tolist())
        test_labels.extend(data.y.detach().cpu().numpy())
        
        if i == 0:
            test_embeddings = modelo.readout
        else:
            test_embeddings = torch.cat((test_embeddings,modelo.readout.detach()), 0)
        
        #print(modelo.emb.shape)
    return loss.detach().cpu(), correct / len(loader.dataset), test_embeddings, test_labels, test_predictions

**Evaluar modelo para chequear comportamiento**

In [ ]:
import numpy as np
loss, acc, test_embeddings, test_labels, test_predictions = test_embedd(model, loader, device)
test_embeddings = test_embeddings.detach().cpu()
print('Embedding shape:',test_embeddings.shape, test_embeddings.device)
print('Dataset size:',len(test_predictions))

In [ ]:
print('Embedding shape:',test_embeddings.shape, test_embeddings.device)
print('Dataset size:',len(test_predictions))

In [ ]:
loss, acc

**Plotear embeddings con sus correspondientes etiquetas y predicciones en sus correspondiente partición.**

In [ ]:
train_indices, test_indices = train_test_split(list(range(len(dataset.data.y))), test_size=0.2, stratify=dataset.data.y,
                                random_state=12345, shuffle=True)
#train_dataset = torch.utils.data.Subset(dataset, train_indices)
#test_dataset = torch.utils.data.Subset(dataset, test_indices)

e = np.array(test_embeddings)
p = np.array(test_predictions)
l = np.array(test_labels)
print('Acc:',(np.sum(p == l)/len(p))*100)

e_train = e[train_indices]
e_test  = e[test_indices]
lab_train = l[train_indices]
lab_test  = l[test_indices]
pred_train = p[train_indices]
pred_test  = p[test_indices]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from matplotlib import cm

def print_embeddings(train_embedd, test_embedd, train_labels, test_labels, train_pred, test_pred, num_categories):
    
    train_embedd = np.array(train_embedd)
    train_labels = np.array(train_labels)
    test_embedd = np.array(test_embedd)
    test_labels = np.array(test_labels)

    tsne = TSNE(n_components=2, verbose=0, perplexity=80, learning_rate=800)
    tsne_results = tsne.fit_transform(np.vstack([train_embedd,test_embedd]))
    
    train_embedd_2dim = tsne_results[:len(train_embedd)]
    test_embedd_2dim = tsne_results[len(train_embedd):]

    cmap = cm.get_cmap('Set2')
    fig, ax = plt.subplots(1, 2, figsize=(17,8))

    for lab in range(num_categories):
        train_indices_lab = train_labels == lab
        test_indices_lab = test_labels == lab
        
        ax[0].scatter(train_embedd_2dim[train_indices_lab,0], train_embedd_2dim[train_indices_lab,1],
                   c=np.array(cmap(lab)).reshape(1,4), label = f"{lab}_train" ,alpha=0.5, marker ='o')
        #ax[0].scatter(test_embedd_2dim[test_indices_lab,0], test_embedd_2dim[test_indices_lab,1],
        #           c=np.array(cmap(lab)).reshape(1,4), label = f"{lab}_test" ,alpha=0.5, marker ='x')
        ax[0].set_title('Real Label')
        
    for lab in range(num_categories):
        train_indices_pred = train_pred == lab
        test_indices_pred = test_pred == lab
        
        acc_test = (np.sum(test_pred == test_labels)/len(test_labels))*100
        acc_train = (np.sum(train_pred == train_labels)/len(train_pred))*100
        
        #ax[1].scatter(train_embedd_2dim[train_indices_pred,0], train_embedd_2dim[train_indices_pred,1],
        #           c=np.array(cmap(lab)).reshape(1,4), label = f"{lab}_train" ,alpha=0.5, marker ='o')
        ax[1].scatter(test_embedd_2dim[test_indices_pred,0], test_embedd_2dim[test_indices_pred,1],
                   c=np.array(cmap(lab)).reshape(1,4), label = f"{lab}_test" ,alpha=0.5, marker ='x')
        ax[1].set_title(f'Predicted\n$Acc_{{train}}={acc_train:.1f}$ - $Acc_{{test}}={acc_test:.1f}$')

        ax[1].legend(fontsize='large', markerscale=2)
    plt.show()

In [ ]:
print_embeddings(e_train, e_test, lab_train, lab_test, pred_train, pred_test, len(set(l)))

In [ ]:
assert 1==0

## Table of percentage of added/removed edges

In [9]:
class CTNet_adj_rw(torch.nn.Module):
    def __init__(self, in_channels, out_channels, k_centers, hidden_channels=32):
        super(CTNet_adj_rw, self).__init__()
    
        self.lin1 = Linear(in_channels, hidden_channels)
        num_of_centers1 =  k_centers # k1 #order of number of nodes
        self.pool1 = Linear(hidden_channels, num_of_centers1)
        #self.CT = CTLayer()
        self.conv1 = DenseGraphConv(hidden_channels, hidden_channels)
        num_of_centers2 =  16 # k2 #mincut 
        self.pool2 = Linear(hidden_channels, num_of_centers2)
        #self.MinCut = MinCutLayer()
        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels) # MLPs towards out 
        self.lin3 = Linear(hidden_channels, out_channels)
        
        self.new_adj = torch.zeros(0)#Creamos la variable que recoge nuestros embedings

    def forward(self, x, edge_index, batch):    # x torch.Size([N, N]),  data.batch  torch.Size([661])  
        # Make all adjacencies of size NxN 
        adj = to_dense_adj(edge_index, batch) # adj torch.Size(B, N, N])
        # Make all x_i of size N=MAX(N1,...,N20), e.g. N=40:
        x, mask = to_dense_batch(x, batch) # x torch.Size([20, N, 32]) ; mask torch.Size([20, N]) batch_size=20

        x = self.lin1(x)
        # First mincut pool for computing Fiedler adn rewire 
        s1  = self.pool1(x)

        if torch.isnan(adj).any():
            print("adj nan")
        if torch.isnan(x).any():
            print("x nan")
        
        # CT REWIRING
        adj, CT_loss, ortho_loss1 = dense_CT_rewiring(x, adj, s1, mask) # out: x torch.Size([20, N, F'=32]),  adj torch.Size([20, N, N])

        self.new_adj = adj.clone()
        # CONV1: Now on x and rewired adj: 
        x = self.conv1(x, adj) #out: x torch.Size([20, N, F'=32])

        # MLP of k=16 outputs s
        s2 = self.pool2(x) # s torch.Size([20, N, k])
        
        # MINCUT_POOL
        x, adj, mincut_loss2, ortho_loss2 = dense_mincut_pool(x, adj, s2, mask) # out x torch.Size([20, k=16, F'=32]),  adj torch.Size([20, k2=16, k2=16])

        # CONV2: Now on coarsened x and adj: 
        x = self.conv2(x, adj) #out x torch.Size([20, 16, 32])
        
        # Readout for each of the 20 graphs
        x = x.sum(dim=1) # x torch.Size([20, 32])
        
        # Final MLP for graph classification: hidden channels = 32
        x = F.relu(self.lin2(x)) # x torch.Size([20, 32])
        x = self.lin3(x) #x torch.Size([20, 2])
        #print(x.shape)
        
        CT_loss = CT_loss + ortho_loss1
        mincut_loss = mincut_loss2 + ortho_loss2
        return F.log_softmax(x, dim=-1), CT_loss, mincut_loss

In [20]:
from torch_geometric.utils import dense_to_sparse, to_dense_adj

@torch.no_grad()
def test_get_pct_modified_edges(modelo, loader, device):
    
    test_predictions = []
    test_labels = []
    added_edges_pct = []
    removed_edges_pct = []
    
    modelo.eval()
    correct = 0
    
    for i,data in enumerate(loader):
        data = data.to(device)
        pred, mc_loss, o_loss = modelo(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()
        test_predictions.extend(pred.max(dim=1)[1].tolist())
        test_labels.extend(data.y.detach().cpu().numpy())
        
        original_adjs = to_dense_adj(data.edge_index, data.batch).detach().cpu().numpy()
        adj_rw = modelo.new_adj.detach().clone().cpu().numpy()
        original = np.count_nonzero(original_adjs, axis=(1,2))
        removed = np.count_nonzero(original_adjs, axis=(1,2))-np.count_nonzero(original_adjs*adj_rw, axis=(1,2))
        added = np.count_nonzero(adj_rw, axis=(1,2)) - (original-removed)
        removed_pct = (removed/original)*100
        added_pct =(added/original)*100
        added_edges_pct.extend(added_pct)
        removed_edges_pct.extend(removed_edges_pct)
        print()
        
        
    return loss.detach().cpu(), correct/len(test_labels), added_edges_pct, removed_edges_pct, test_labels, test_predictions

In [21]:
model =  CTNet_adj_rw(dataset.num_features, dataset.num_classes, k_centers=num_of_centers).to(device)
model.load_state_dict(torch.load(modelito, map_location=torch.device(device)))
model.eval()

CTNet_adj_rw(
  (lin1): Linear(in_features=1, out_features=32, bias=True)
  (pool1): Linear(in_features=32, out_features=420, bias=True)
  (conv1): DenseGraphConv(32, 32)
  (pool2): Linear(in_features=32, out_features=16, bias=True)
  (conv2): DenseGraphConv(32, 32)
  (lin2): Linear(in_features=32, out_features=32, bias=True)
  (lin3): Linear(in_features=32, out_features=2, bias=True)
)

In [22]:
loss, acc, edge_aditions, edge_removals, test_labels, test_predictions = test_get_pct_modified_edges(model, loader, device)
adj_rw = adj_rw.detach().cpu()
print('ADJ shape:',adj_rw.shape, adj_rw.device)
print('Dataset size:',len(test_predictions))
acc

RuntimeError: CUDA out of memory. Tried to allocate 236.00 MiB (GPU 0; 6.00 GiB total capacity; 4.58 GiB already allocated; 0 bytes free; 4.66 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
p = np.array(test_predictions)
l = np.array(test_labels)
print('Acc:',(np.sum(p == l)/len(p))*100)

In [ ]:
np.mean(edge_aditions), np.std(edge_aditions)

In [ ]:
plt.hist(edge_aditions, bins=int(np.sqrt(len(edge_aditions))));

In [ ]:
np.mean(edge_removals), np.std(edge_removals)

In [ ]:
plt.hist(edge_removals, bins=int(np.sqrt(len(edge_removals))));

## Rewiring plots and

In [ ]:
@torch.no_grad()
def test_rewiring(modelo, loader, device): #get rewired adjacency from one graph (pass batch)
    test_predictions = []
    test_labels = []
    modelo.eval()
    correct = 0
    for i,data in enumerate(loader):
        data = data.to(device)
        pred, mc_loss, o_loss = modelo(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()
        test_predictions.extend(pred.max(dim=1)[1].tolist())
        test_labels.extend(data.y.detach().cpu().numpy())
        
        print(modelo.new_adj.shape)
        if i == 0:
            test_adj = modelo.new_adj
        else:
            pass
            #test_adj = torch.cat((test_adj,modelo.new_adj.detach()), 0)
        
        #print(modelo.emb.shape)
    return loss.detach().cpu(), correct/len(test_labels), test_adj, test_labels, test_predictions

In [ ]:
model =  CTNet_adj_rw(dataset.num_features, dataset.num_classes, k_centers=num_of_centers).to(device)
model.load_state_dict(torch.load(modelito, map_location=torch.device(device)))
model.eval()

Sacamos rewire del batch que queramos

In [ ]:
loss, acc, adj_rw, test_labels, test_predictions = test_rewiring(model, [next(iter(loader))], device)
adj_rw = adj_rw.detach().cpu()
print('ADJ shape:',adj_rw.shape, adj_rw.device)
print('Dataset size:',len(test_predictions))
acc

In [ ]:
p = np.array(test_predictions)
l = np.array(test_labels)
print('Acc:',(np.sum(p == l)/len(p))*100)

Miramos donde ha acertado

Lo pasamos a nodos

Percentage of removed and added graphs

In [ ]:
def diff_adj(adj1, adj2):
    assert adj1.shape == adj2.shape
    added = 0
    removed = 0
    mantained = 0
    original = np.count_nonzero(adj1)
    for i in range(adj1.shape[0]):
        for j in range(adj1.shape[1]):
            
            if adj1[i,j]==1 and adj2[i,j]==0:
                removed+=1
                
            elif adj1[i,j]==0 and adj2[i,j]==1:
                added+=1
            elif adj1[i,j]==1 and adj2[i,j]==1:
                mantained+=1
                
    return added, removed, mantained, original

In [ ]:
eps=0

In [ ]:
adj_rw[adj_rw<eps] = 0
adj_rw[adj_rw!=0] = 1
adj_rw.shape, np.count_nonzero(adj_rw)

In [ ]:
batch_original_adj = to_dense_adj(next(iter(loader)).edge_index, next(iter(loader)).batch)
batch_original_adj.shape, np.count_nonzero(batch_original_adj)

In [ ]:
np.count_nonzero(batch_original_adj[0]), np.count_nonzero(adj_rw[0])
                

In [ ]:
added, removed, mantained, original = diff_adj(batch_original_adj[5], adj_rw[5])
added, removed, mantained, original

In [ ]:

original2

Para ver solo uno - Recableado

In [ ]:
dense_to_sparse(adj_rw.squeeze(0))

## All graphs embedding

# Embeddings GapNet Laplacian

In [ ]:
test_loader =  DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
test_loader =  DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
model =  GAPNet(dataset.num_features, dataset.num_classes, derivative="laplacian",device=device)
model.load_state_dict(torch.load("models/REDDIT-BINARY_GAPNet_laplacian_iter0.pth", map_location=torch.device('cpu')))
model.eval()

# Statistical data about the graph

In [ ]:
import networkx as nx
from torch_geometric import utils
import numpy as np
from tqdm.notebook import tqdm


nxdata = {
    'Assortativity':np.empty(len(dataset)),
    'Triangles':np.empty(len(dataset)),
    'Transitivity':np.empty(len(dataset)),
    'Clustering':np.empty(len(dataset))
}
for i, graph in enumerate(tqdm(dataset, desc='Datasets')):
    G = utils.to_networkx(graph, to_undirected=True)
    nxdata['Assortativity'][i] = nx.degree_assortativity_coefficient(G)
    nxdata['Triangles'][i] = np.sum(list(nx.triangles(G).values()))/3
    nxdata['Transitivity'][i] = nx.transitivity(G)
    nxdata['Clustering'][i] = nx.average_clustering(G)


In [ ]:
for k in nxdata:
    print(f"{k}: {nxdata[k].mean():.4f} +- {nxdata[k].std():.4f}")

# Stratify

In [ ]:
from sklearn.model_selection import train_test_split

def class_distrib(dataset):
    d = dict()
    for data in dataset:
        d[int(data.y.numpy())] = d.get(int(data.y.numpy()),0) + 1
    return d

def batch_class_distrib(batch):
    d = dict()
    for label in batch.y:
        d[int(label.numpy())] = d.get(int(label.numpy()),0) + 1
    return d

len(dataset.data.y)

In [ ]:
train_indices, val_indices = train_test_split(list(range(len(dataset.data.y))), test_size=0.2, stratify=dataset.data.y)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

In [ ]:
class_distrib(dataset), class_distrib(train_dataset), class_distrib(val_dataset)

**Optional**

In [ ]:
from torch.utils.data import SequentialSampler

In [ ]:
class StratifiedSampler():
    """Stratified Sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, class_vector, batch_size):
        """
        Arguments
        ---------
        class_vector : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(class_vector.size(0) / batch_size)
        self.class_vector = class_vector

    def gen_sample_array(self):
        try:
            from sklearn.model_selection import StratifiedShuffleSplit
        except:
            print('Need scikit-learn for this functionality')
        import numpy as np
        
        s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.5)
        X = torch.randn(self.class_vector.size(0),2).numpy()
        y = self.class_vector.numpy()
        s.get_n_splits(X, y)

        train_index, test_index = next(s.split(X, y))
        return np.hstack([train_index, test_index])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return len(self.class_vector)

In [ ]:
sampler = StratifiedSampler(class_vector=dataset.data.y, batch_size=BATCH_SIZE)


In [ ]:
loader = DataLoader(dataset, batch_size=2, #train_dataset
                        shuffle=False, sampler=sampler)

## DIGL

In [ ]:
import scipy.sparse as sp
from torch_geometric.utils.convert import to_scipy_sparse_matrix, from_scipy_sparse_matrix
from torch_geometric.utils import dense_to_sparse

In [ ]:
from torch_geometric.transforms import BaseTransform

class DIGLedges(BaseTransform):
    def __init__(self, alpha:float, eps:float, use_edge_weigths = False):
        self.alpha = alpha
        self.eps = eps
        self.use_edge_weigths = use_edge_weigths

    def __call__(self, data):
        new_edges, new_weights = self.digl_edges(data.edge_index, self.alpha, self.eps)
        data.edge_index = new_edges
        
        if self.use_edge_weigths:
            data.edge_weight = new_weights
            
        return data
    
    
    def gdc(self, A: sp.csr_matrix, alpha: float, eps: float):
        N = A.shape[0]

        # Self-loops
        A_loop = sp.eye(N) + A

        # Symmetric transition matrix
        D_loop_vec = A_loop.sum(0).A1
        D_loop_vec_invsqrt = 1 / np.sqrt(D_loop_vec)
        D_loop_invsqrt = sp.diags(D_loop_vec_invsqrt)
        T_sym = D_loop_invsqrt @ A_loop @ D_loop_invsqrt

        # PPR-based diffusion
        S = alpha * sp.linalg.inv(sp.eye(N) - (1 - alpha) * T_sym)

        # Sparsify using threshold epsilon
        S_tilde = S.multiply(S >= eps)

        # Column-normalized transition matrix on graph S_tilde
        D_tilde_vec = S_tilde.sum(0).A1
        T_S = S_tilde / D_tilde_vec

        return T_S

    def digl_edges(self, edges, alpha, eps):
        A0 = sp.csr_matrix(to_scipy_sparse_matrix(edges))
        new_sp_matrix = sp.csr_matrix(gdc(A0, self.alpha, self.eps))
        new_edge_index, weights = from_scipy_sparse_matrix(new_sp_matrix)
        return new_edge_index, weights
    
    
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.alpha})'
    
    

In [ ]:
digl_data = TUDataset(root='data/digl/TUDataset',name='MUTAG', transform=DIGLedges(0.5, 0.02))

In [ ]:
digl_data[0].edge_index

In [ ]:
dataset[0].edge_index

In [ ]:
digl_data = TUDataset(root='data/digl/TUDataset',name='MUTAG')

In [ ]:
digl_data[0].edge_index

# Plot result table

In [ ]:
import pandas as pd
import os
table_dict_acc = dict()
table_dict_std = dict()
for file in os.listdir('logs_stratified'):
    data, model, *_ = file.split('_')
    
    f = open('logs_stratified/' + file, 'r')
    for last_line in f:
        pass
    
    result = last_line.split()
    if data not in table_dict_acc:
        table_dict_acc[data]=dict()
        table_dict_std[data]=dict()
    try:
        acc = float(result[0])
        std = float(result[-1])
        table_dict_acc[data][model]=acc
        table_dict_std[data][model]=std
    except:
        print(model, data, "FAIL")
    
    f.close()
    
pd.DataFrame(table_dict)

In [ ]:
df_acc= pd.DataFrame(table_dict_acc).sort_index(ascending=True)*100
df_std= pd.DataFrame(table_dict_std).sort_index(ascending=True)*100

In [ ]:
df_acc

In [ ]:
df_std

In [ ]:
print(f'{"":>11}', end='&')
for dataset in df_acc.columns:
    print(f'{dataset:>17}', end=' &')
print()
for model, values in df_acc.iterrows():
    print(f'{model:>10}', end=' & ')
    for dataset in values.index:
        print(f'${values[dataset]:>5.2f} \\pm {df_std.loc[model, dataset]:>4.2f}$', end=' & ')
    print(r'\\')

# Community layout

In [ ]:
from torch_geometric import utils
import networkx as nx
from community import community_louvain, generate_dendrogram, partition_at_level

def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=1.)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

def test():
    # to install networkx 2.0 compatible version of python-louvain use:
    # pip install -U git+https://github.com/taynaud/python-louvain.git@networkx2
    from community import community_louvain

    g = nx.karate_club_graph()
    
    dendrogram = generate_dendrogram(g)
    partition = partition_at_level(dendrogram, len(dendrogram) - 1)
    
    #partition = community_louvain.best_partition(g)
    pos = community_layout(g, partition)

    nx.draw(g, pos, node_color=list(partition.values())); plt.show()
    return

In [ ]:
test()

In [ ]:
G = utils.to_networkx(dataset[20], to_undirected=True)


In [ ]:
partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
nx.draw(G, pos, node_color=list(partition.values())); plt.show()

https://stackoverflow.com/questions/43541376/how-to-draw-communities-with-networkx

In [ ]:
np.array(cmap(3)).reshape(1,4)

In [ ]:
cmap = cm.get_cmap('Set2')

In [ ]:
cmap(3)

In [ ]:
node_color = {node: cmap(community_id) for node, community_id in partition.items()}


In [ ]:
node_color

In [ ]:
from netgraph import Graph
# https://netgraph.readthedocs.io/en/latest/graph_classes.html#netgraph.Graph

In [ ]:
plt.figure(figsize=(8, 6))
Graph(G, node_color=node_color,node_size=1,
      node_edge_width=0, edge_alpha=0.1,
      node_layout='community', node_layout_kwargs=dict(node_to_community=partition),
      edge_layout='bundled', edge_layout_kwargs=dict(k=2000),
) # edge_cmap!!!!! #edge_color
plt.show()